Autor: Manuel Meniño Magán

Proyecto: 11MBIG TFM. Detección de anomalías en series temporales de tráfico de Internet. Aplicación en indicadores de red de Vodafone España

Descripción: En este cuaderno se realiza la carga de los ficheros de SID para obtener los ficheros de entrenamiento que se usarán posteriormente

##Librerias

In [2]:
import requests
import io
import pandas_profiling
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from google.colab import files
from google.colab import drive
import datetime
import time

##Carga de datos

Carga de ficheros en local

In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
data = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), names=['timeStamp', 'elementId', 'serviceId', 'typeId', 'value'])

Carga de ficheros de Google drive

In [3]:
drive.mount('/gdrive')

Mounted at /gdrive


In [7]:
#El fichero SID_DATA_2020-05to08.csv es una concatenación todos los ficheros diarios de SID disponibles, desde el 08/05/2020 hasta 13/08/2020
!cp '/gdrive/My Drive/11MBIG TFM/SID Data/SID_DATA_2020-05to08.csv' data.csv
!ls -la | grep .csv

-rw------- 1 root root 1884497317 Oct  2 09:28 data.csv


In [27]:
data = pd.read_csv('data.csv', names=['timeStamp', 'elementId', 'serviceId', 'typeId', 'value'])

##Exploración previa

In [28]:
data.head()

,timeStamp,elementId,serviceId,typeId,value
0,15889668,202,9,1,NaN
1,15889668,203,9,1,NaN
2,15889668,200,9,1,NaN
3,15889668,201,9,1,NaN
4,15889668,206,9,1,NaN


In [29]:
data.tail()

,timeStamp,elementId,serviceId,typeId,value
56096194,15973011,202,10,1,0.000000e+00
56096195,15973011,207,10,1,0.000000e+00
56096196,15973011,206,10,1,0.000000e+00
56096197,15973011,201,10,1,1.002000e+09
56096198,15973011,200,10,1,1.001952e+09


In [30]:
data.dtypes

timeStamp      int64
elementId      int64
serviceId      int64
typeId         int64
value        float64
dtype: object

In [31]:
data.isnull().sum()

timeStamp        0
elementId        0
serviceId        0
typeId           0
value        48573
dtype: int64

##Transformación y limpieza

In [ ]:
#Se multiplica el timestamp por 100 ya que en SID no se guardan los 2 últimos ceros.
data['timeStamp'] = 100 * data['timeStamp']

In [33]:
#Se pone el campo tiemstamp cómo índice
data.set_index(keys='timeStamp', inplace=True)
data.head()

,elementId,serviceId,typeId,value
timeStamp,,,,
1588966800,202,9,1,NaN
1588966800,203,9,1,NaN
1588966800,200,9,1,NaN
1588966800,201,9,1,NaN
1588966800,206,9,1,NaN


Se selecciona con un equipo de red (elementId) y un servicio (serviceId) concretos

In [34]:
elemento = 9 #Corresponde al CdC de Vigo equipo1
servicio = 1 # Corresponde al servicio de Internet
data_element009_service01 = data[(data['elementId']==elemento) & (data['serviceId']==servicio)]
#Nos quedamos con la columna value solamente
data_element009_service01.drop(columns = ['elementId', 'serviceId', 'typeId'], inplace=True)
data_element009_service01.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,value
timeStamp,
1588966800,NaN
1588967100,3.013337e+10
1588967400,3.005537e+10
1588967700,3.027482e+10
1588968000,3.172016e+10


In [35]:
data_element009_service01.isnull().sum()

value    14
dtype: int64

In [36]:
#Se reemplazan los nulos por el valor inmediatamente anterior (no funciona si el prmer valor de la serie es nulo)
data_element009_service01['value'].fillna(method = 'ffill', inplace=True)
data_element009_service01.isnull().sum()

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


value    1
dtype: int64

In [37]:
elemento = 10 #Corresponde al CdC de Vigo equipo2
servicio = 1 # Corresponde al servicio de Internet
data_element010_service01 = data[(data['elementId']==elemento) & (data['serviceId']==servicio)]
#Nos quedamos con la columna value solamente
data_element010_service01.drop(columns = ['elementId', 'serviceId', 'typeId'], inplace = True)
data_element010_service01.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,value
timeStamp,
1588966800,NaN
1588967100,2.667760e+10
1588967400,2.458522e+10
1588967700,2.767156e+10
1588968000,2.713109e+10


In [38]:
data_element010_service01.isnull().sum()

value    14
dtype: int64

In [39]:
data_element010_service01['value'].fillna(method = 'ffill', inplace=True)
data_element010_service01.isnull().sum()

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


value    1
dtype: int64

Se suman los valores de ambos equipos para obtener el valor conjunto del CdC de Vigo (cdcId = 5)

In [40]:
data_cdc05_service01 = pd.merge(data_element009_service01, data_element010_service01, on='timeStamp', suffixes=('_1', '_2'))
data_cdc05_service01['total'] = data_cdc05_service01.loc[:, 'value_1'] + data_cdc05_service01.loc[:, 'value_2']
data_cdc05_service01.head()

,value_1,value_2,total
timeStamp,,,
1588966800,NaN,NaN,NaN
1588967100,3.013337e+10,2.667760e+10,5.681097e+10
1588967400,3.005537e+10,2.458522e+10,5.464059e+10
1588967700,3.027482e+10,2.767156e+10,5.794638e+10
1588968000,3.172016e+10,2.713109e+10,5.885125e+10


Los valores de tráfico están en bps se recalcula para Gbps

In [41]:
data_cdc05_service01['value_1'] = data_cdc05_service01['value_1']/(1024*1024*1024)
data_cdc05_service01['value_2'] = data_cdc05_service01['value_2']/(1024*1024*1024)
data_cdc05_service01['total'] = data_cdc05_service01['total']/(1024*1024*1024)
data_cdc05_service01.head()

,value_1,value_2,total
timeStamp,,,
1588966800,NaN,NaN,NaN
1588967100,28.063890,24.845453,52.909342
1588967400,27.991246,22.896772,50.888018
1588967700,28.195625,25.771149,53.966774
1588968000,29.541703,25.267798,54.809501


Se guarda el fichero en el drive

In [42]:
data_cdc05_service01.to_csv('/gdrive/My Drive/11MBIG TFM/SID Data/data_cdc05_service01_2020-05to08.csv', columns=['value_1', 'value_2', 'total'])

##Selección de sets de train y test

Variables temporales en relación a las muestras

In [55]:
epoch_week27_start=1593381600
epoch_week28_start=1593986400
epoch_week29_start=1594591200
epoch_week30_start=1595196000
epoch_week31_start=1595800800
epoch_week32_start=1596405600
epoch_hour = 60*60
epoch_day = 24 * epoch_hour
epoch_week = 7 * epoch_day
sid_muestras_time = 300 #300 sg = 5 min. por ser el valor entre muestras de SID
day_samples = int(epoch_day/sid_muestras_time)
week_samples = int(epoch_week/sid_muestras_time)

Nos quedamos con el campo total

In [57]:
serie = data_cdc05_service01.loc[:, 'total']
serie.head()

timeStamp
1593381600    55.950883
1593381900    55.184399
1593382200    54.702931
1593382500    53.737060
1593382800    51.766890
Name: total, dtype: float64

Split train y test

In [58]:
serie_train = serie[serie.index >= epoch_week27_start]
serie_train = serie_train[serie.index < epoch_week31_start]
serie_train.head()

timeStamp
1593381600    55.950883
1593381900    55.184399
1593382200    54.702931
1593382500    53.737060
1593382800    51.766890
Name: total, dtype: float64

In [59]:
serie_train.tail()

timeStamp
1595799300    59.284297
1595799600    58.385666
1595799900    58.152342
1595800200    57.767017
1595800500    56.195818
Name: total, dtype: float64

In [60]:
serie_test = serie[serie.index >= epoch_week31_start]
serie_test = serie_test[serie_test.index < epoch_week32_start]
serie_test.head()

timeStamp
1595800800    55.256671
1595801100    55.490521
1595801400    53.201406
1595801700    52.884181
1595802000    52.357252
Name: total, dtype: float64

In [61]:
serie_test.tail()

timeStamp
1596404100    59.931283
1596404400    58.796034
1596404700    58.127769
1596405000    58.340047
1596405300    58.463460
Name: total, dtype: float64

Exportar datasets train y test a csv

In [63]:
serie_train.to_csv('/gdrive/My Drive/11MBIG TFM/SID Data/train_inet_vigo_202007.csv')
serie_test.to_csv('/gdrive/My Drive/11MBIG TFM/SID Data/test_inet_vigo_202007.csv')